# Export 2 videos for youtube

In [1]:
import sys
import os
import importlib
import pymongo
from dotenv import load_dotenv
# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
load_dotenv(override=True) 
import const
FF_FULL_PATH = os.environ['FF_FULL_PATH']

In [3]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']

In [4]:
import exam_settings
importlib.reload(exam_settings) #always get latest code

<module 'exam_settings' from '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/exam_settings/__init__.py'>

In [5]:
def main(num_of_questions, cert_symbol, channel):
    cert_metadata = exam_settings.get_cert_metadata(db, cert_symbol)
    questions = exam_settings.query_random_questions(db, num_of_questions, channel, cert_metadata)
    #print(questions)
    today_yyyymmdd = const.get_current_date_yyyymmdd()
    #export to images
    exported_result = exam_settings.generate_images(today_yyyymmdd, cert_metadata, questions)
    if exported_result == False:
        print('Cannot export all images')
        return
    #create video for questions
    exported_result = exam_settings.create_video_from_images(FF_FULL_PATH, cert_metadata['img_folder_path']+"/"+today_yyyymmdd, today_yyyymmdd+".mp4", "1/30", "/%02d.png")
    if exported_result == False:
        print('Cannot export video for questions')
        return
    #create video for answers
    exported_result = exam_settings.create_video_from_images(FF_FULL_PATH, cert_metadata['img_folder_path']+"/"+today_yyyymmdd, today_yyyymmdd+"_explan.mp4", "1/5", "/_%02d.png")
    if exported_result == False:
        print('Cannot export video for explanations')
        return
    #update db
    exam_settings.update_questions_posted(db, cert_metadata, channel, today_yyyymmdd, questions)



In [ ]:
#AWS_SAA, AWS_SAP, AWS_CLF_C02, AWS_DVA_C02, AWS_MLA
#main(20, 'AWS_DVA_C02', 'Youtube')  #export 1 video with 20 images (30 secs per each), total ~ 5min 20 secs

[8181:125839:0625/215206.979491:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[8187:125849:0625/215207.408590:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8187:125849:0625/215207.414273:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8187:125849:0625/215207.414917:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8187:125849:0625/215207.415501:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8187:125849:0625/215207.490517:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8187:125849:0625/215207.501023:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8187:125849:0625/215207.679536:ERROR:ui/gl/gl_display.cc:508] EGL Driver messa

Done generating 1 page image


[8208:126119:0625/215209.002118:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[8212:126129:0625/215209.439799:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8212:126129:0625/215209.445600:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8212:126129:0625/215209.446197:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8212:126129:0625/215209.446739:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8212:126129:0625/215209.519674:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8212:126129:0625/215209.530798:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8212:126129:0625/215209.723657:ERROR:ui/gl/gl_display.cc:508] EGL Driver messa

Done generating 1 page image


[8227:126357:0625/215211.027148:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[8231:126376:0625/215211.377188:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8231:126376:0625/215211.382428:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8231:126376:0625/215211.383045:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8231:126376:0625/215211.383572:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8231:126376:0625/215211.444886:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8231:126376:0625/215211.452536:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8231:126376:0625/215211.636160:ERROR:ui/gl/gl_display.cc:508] EGL Driver messa

...

Done generating 1 page image


[8943:134885:0625/215320.209617:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[8947:134890:0625/215320.562062:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8947:134890:0625/215320.567789:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8947:134890:0625/215320.569010:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8947:134890:0625/215320.569580:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8947:134890:0625/215320.632956:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8947:134890:0625/215320.641717:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8947:134890:0625/215320.828232:ERROR:ui/gl/gl_display.cc:508] EGL Driver messa

Done generating 1 page image


[8961:135104:0625/215321.725315:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[8965:135125:0625/215322.137209:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8965:135125:0625/215322.142879:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8965:135125:0625/215322.144074:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8965:135125:0625/215322.144552:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8965:135125:0625/215322.199300:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8965:135125:0625/215322.206966:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[8965:135125:0625/215322.373922:ERROR:ui/gl/gl_display.cc:508] EGL Driver messa

Done generating 1 page image
21
Attempting to create video from images in: /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/aws_dev_asso/img_youtube/20250625
Output video will be: /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/aws_dev_asso/img_youtube/20250625/20250625.mp4
FFmpeg command: /Users/sang/Downloads/SetupFiles/ffmpeg/ffmpeg -framerate 1/30 -i /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/aws_dev_asso/img_youtube/20250625/%02d.png -c:v libx264 -r 25 -pix_fmt yuv420p -y /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/aws_dev_asso/img_youtube/20250625/20250625.mp4

FFmpeg command executed successfully!
STDOUT:

STDERR:
ffmpeg version 7.1.1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-li